In [1]:
import evaluate
import transformers
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DistilBertForSequenceClassification,
)

In [2]:
"""
Load dataset.
"""
dataset = load_dataset("yelp_review_full")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [3]:
"""
Initialize tokenizer and model.
"""
model_id = "distilbert-base-uncased"

# init tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# init model
model = DistilBertForSequenceClassification.from_pretrained(
    model_id,
    num_labels=5,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
"""
Tokenize dataset.
"""
def tokenize(X):
    return tokenizer(
        X["text"],
        padding="max_length",
        truncation=True,
    )

# tokenize data
tokenized_datasets = dataset.map(tokenize, batched=True)
print(tokenized_dataset)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

In [ ]:
"""
Create dataset splits.
"""
seed = 777
n_samples = 1000

train_dataset = tokenized_datasets["train"].shuffle(seed=seed).select(range(n_samples))
eval_dataset = tokenized_datasets["test"].shuffle(seed=seed).select(range(0, n_samples))
test_dataset = tokenized_datasets["test"].shuffle(seed=seed).select(range(n_samples, 2 * n_samples))

In [ ]:
"""
Create Trainer.
"""
# define metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# training args
training_args = TrainingArguments(
    output_dir="results",
    eval_strategy="epoch",
)

# init trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
"""
Train model.
"""
trainer.train()

In [ ]:
"""
Evaluate trained model.
"""
trainer.evaluate(test_dataset)